In [346]:
# -*- coding: utf-8 -*-
"""
Created on Mon Nov 18 11:58:32 2019

@author: Liduino Pitombeira
"""

from music21 import *
import numpy as np
import pandas as pd


class MusicaIn:
    
    def __init__(self,filename,partes,tam):
        self.filename = filename
        self.partes = partes
        self.tam = tam
        self.dados_alturas = None
        self.dados_duracoes = None
        self.tom = None
        self.modo = None


    #Extract the pitch parameter from a MIDI file

    def get_dados(self):
        piece = converter.parse(self.filename)
        dados_alturas = []
        dados_duracoes = []
        escala = piece.analyze('key')
        self.tom = escala.getTonic().midi % 12
        self.modo = escala.mode
        if self.partes == None:
            self.partes = list(range(len(piece)))
        if self.tam==None:
            self.tam = -1
            
        for j in range(len(piece)):
            if j in self.partes: #indice das partes que serão analisadas
                for event in piece.flat.notes[:self.tam]: #tam define quantas notas serão analisadas
                    if type(event) == note.Note:
                        evento_altura = event.pitch.midi
                        evento_duracao = float(event.duration.quarterLength)
                        dados_alturas.append(evento_altura)
                        if evento_duracao != 0.0 : 
                            dados_duracoes.append(evento_duracao)

                    #caso de tempo de fazer para acordes
                    elif type(event) == chord.Chord:
                        normalOrder = event.normalOrder
                        tonica = normalOrder[0]
                        dados_alturas.append(tonica)
    
        self.dados_alturas = np.array(dados_alturas)
        self.dados_duracoes = np.array(dados_duracoes)
        

class MusicaOut:
    
    def __init__(self,filename,instrumentos,tam,escala,oitavas,duracoes,modo_sustain):
        self.filename = filename
        self.instrumentos = instrumentos 
        self.tam = tam
        dic_escala = {'c':0,'c#':1,'d':2,'e-':3,'e':4,'f':5,'f#':6,'g':7,'a-':8,'a':9,'b-':10,'b':11}
        self.tom = dic_escala[escala.split(' ')[0].lower()]
        self.modo = escala.split(' ')[1]
        self.oitavas = oitavas
        self.duracoes_mult = duracoes_mult
        self.modo_sustain = modo_sustain
        self.dados_alturas = []
        self.dados_duracoes = []
        self.M_alturas = []
        self.M_duracoes = []
        self.extra = None
        
    def normalize(self,matriz):
        for i in range(len(matriz)):
            total = sum(matriz[i])
            for j in range(len(matriz[i])):
                matriz[i][j] = matriz[i][j]/total
                
        return np.array(matriz)
    
    def transpose(self,pitches,old_tom,new_tom,old_modo,new_modo):

        if new_tom == old_tom:
            intervalo = 0
        else:
            intervalo1 = new_tom-old_tom

            intervalo2 = intervalo1 +12*(intervalo1/abs(intervalo1))

            if abs(intervalo1)<abs(intervalo2):
                intervalo = intervalo1
            else:
                intervalo = intervalo2

        if (old_modo,new_modo) == ('major','major') or (old_modo,new_modo) == ('minor','minor'):
            return [pitch + intervalo for pitch in pitches]
        elif (old_modo,new_modo) == ('major','minor'):
            return [pitch +intervalo+3 for pitch in pitches]

        elif (old_modo,new_modo) == ('minor','major'):
            return [pitch +intervalo-3 for pitch in pitches]
            
    def set_dados(self,musicas):
        
        for mus in musicas:
            self.dados_alturas += self.transpose(mus.dados_alturas,mus.tom,self.tom,mus.modo,self.modo)
            self.dados_duracoes  += dados_duracoes
        
    def set_matriz_de_transição(self):

        #List with unique values of A and in ascending order 
        Aset_alturas = list(set(self.dados_alturas))
        Aset_alturas.sort()
         #List with unique values of A and in ascending order 
        Aset_duracoes = list(set(self.dados_duracoes))
        Aset_duracoes.sort()

        #List of list filled with zeros (this will be filled by the iteractions)
        M_alturas = np.zeros((len(Aset_alturas),len(Aset_alturas))).astype(np.float) 
        M_duracoes = np.zeros((len(Aset_duracoes),len(Aset_duracoes))).astype(np.float) 

        #Gera a matriz de transição de proababilidades

        for alt_row,alt_col in zip(self.dados_alturas[:-1],self.dados_alturas[1:]):
            pos_row = Aset_alturas.index(alt_row)
            pos_col = Aset_alturas.index(alt_col)
            #print(alt_row,alt_col,pos_row,pos_col)
            M_alturas[pos_row,pos_col] += 1

        for row,col in zip(self.dados_duracoes[:-1],self.dados_duracoes[1:]):
            pos_row = Aset_duracoes.index(row)
            pos_col = Aset_duracoes.index(col)
            M_duracoes[pos_row,pos_col] += 1

        #Normaliza a matriz
        M_alturas = self.normalize(list(M_alturas))
        M_duracoes = self.normalize(list(M_duracoes))
        self.M_alturas = M_alturas
        self.M_duracoes = M_duracoes
    
    def gerar_nova_melodia(self):
        #Sorteando através da matriz de transição

        #Valor inicial
        Bset_alturas = list(set(self.dados_alturas))
        Bset_alturas.sort()
        Bset_duracoes = list(set(self.dados_duracoes))
        Bset_duracoes.sort()
        altura = np.random.choice(Bset_alturas, 1, True)
        duracao = np.random.choice(Bset_duracoes,1, True)
        print(f'nota sorteada inicial = {altura} com duração = {duracao}\n')

        alturas = []
        duracoes = []
        
        for i in range(len(instrumentos)):
            alturas_instrumento = []
            duracoes_instrumento = []
            for i in range(self.tam):
                alturas_instrumento.append(altura[0])
                prob_alturas = self.M_alturas[Bset_alturas.index(altura)]
                prob_alturas = prob_alturas/prob_alturas.sum() #garantir que a soma dê 1.0
                altura = np.random.choice(Bset_alturas, 1, True, prob_alturas)
                duracoes_instrumento.append(duracao[0])
                prob_duracoes = self.M_duracoes[Bset_duracoes.index(duracao)]
                prob_duracoes = prob_duracoes/prob_duracoes.sum()
                duracao = np.random.choice(Bset_duracoes,1,True,prob_duracoes)
            alturas.append(alturas_instrumento)
            duracoes.append(duracoes_instrumento)
        
        #print(f'Nova melodia = {list(zip(alturas,duracoes))}\n')


        melodia = stream.Stream()
        
        
        for i in range(len(instrumentos)):
            instrumento = instrumentos[i]
            duracao_mult = self.duracoes_mult[i]
            parte = stream.Part()
            parte.insert(0,instrumento)
            alturas_parte = alturas[i]
            duracoes_parte = duracoes[i]
            for altura,duracao in list(zip(alturas_parte,duracoes_parte)):
                oitava = np.random.choice(self.oitavas[i],1,True)[0]
                altura = (oitava+1)*12+altura%12  
                nota = note.Note(altura)
                if self.modo_sustain == 'varia':
                    sustain = np.random.choice([True,False],1,True)[0]
                    if sustain:
                        nota.quarterLength = duracao*duracao_mult
                    else:
                        nota.quarterLength = 0.1
                        parte.append(nota)
                        nota = note.Rest()
                        nota.quarterLength = duracao*duracao_mult
                        
                elif self.modo_sustain == 'on':
                    nota.quarterLength = duracao*duracao_mult
                
                else:
                    nota.quarterLength = 0.1
                    nota.quarterLength = 0.1
                    parte.append(nota)
                    nota = note.Rest()
                    nota.quarterLength = duracao*duracao_mult
                    
                parte.append(nota)                      

            melodia.append(parte)
        
        melodia.write('midi', self.filename)
        
    def rotina(self,musicas):
        self.set_dados(musicas)
        self.set_matriz_de_transição()
        self.gerar_nova_melodia()

# Fusão das peças "Croatian Rhapsody" com "Great Balls of Fire"

In [ ]:
croat = MusicaIn('midis/Croatian Rhapsody.mid',[1,2,4],None)
balls = MusicaIn('midis/Great Balls of Fire.mid',list(range(7))+[8,9],None)
croat.get_dados()
balls.get_dados()
instrumentos = [instrument.Piano(),instrument.ElectricBass()]
oitavas = [[3,4,5],[2,3]] #seta as oitavas de cada parte/instrumento
duracoes_mult = [3,7] #seta um multiplicador para a duração das notas de cada parte
novamus = MusicaOut('midis/Croatian Balls of Rhapsody.mid',instrumentos,300,'B- major',oitavas,duracoes_mult,'varia')
musicas = [croat,balls]
novamus.rotina(musicas)

# Extra

## Fusão das 4 estações de Vivaldi

In [349]:
import os
musicas = []
for arq in os.listdir('extra/midis'):
    musica = MusicaIn(f'extra/midis/{arq}',None,1000)
    musica.get_dados()
    musicas.append(musica)
    
instrumentos = [instrument.Violin(),instrument.Violoncello()]
duracoe_mult = [0.5,3]
oitavas = [[4,5,6],[2,3,4]]
estacoes = MusicaOut('extra/midis/Estações.mid',instrumentos,600,'C major',oitavas,duracoes_mult,'varia')
estacoes.rotina(musicas)

nota sorteada inicial = [31] com duração = [0.5]

